# Price Etsymator: Exploratory Data Analysis

In this exploration we've made some interesting observations about our dataset, did some cleanup, and have a good sense of the next steps for our exploration. To do this we took the following steps:

* Got rid of the 'category' and 'dispatched_from' columns
* Edited our columns so they're the correct data types 
* Dropped NaN records for price
* Ensured we got rid of records that weren't meeting certain value parameters (i.e. negative delivery time)
* Explored our 'Star_Rating' column to confirm the NaNs were warranted
* Explored the features of top sellers
* Graphed relationships between variables
* Created a correlation heatmap to see the relationship between price and our numeric variables

In the `02_Feature_Engineering` notebook, we'll work through the following steps:

* Do a bit more data-cleaning, including eliminating the 1* rated listings that are throwing off our info
* Get a sense of the trends we've seen here on a shop-level rather than a listing-level. 
* Create a 'total revenue' feature for each shop (take the price * the number of sales) 
    * We'll explore this new feature by looking at how the mean numeric features vary between the highest revenue earners and the average revenue earners.
* Create a 'total price' column which is a combination of price and cost_delivery - we'll get a sense of whether there are any trends here, though ultimately may not make sure of it.
* Create new features from our object data (title and description) - we'll determine the right keywords to mine for and create those columns 
* Create two dataframes to account for the different ways we can approach the star_rating NaNs - in one we'll impute the NaNs with zeroes, and in the other we'll drop the records that are NaNs.

In [1]:
#import all packages 

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import missingno as msno
import re
from matplotlib.pyplot import figure

In [2]:
#read in our raw data and print out the head

df = pd.read_csv('feature_engineering.csv', index_col=0)

df.head()

,Title,Shop_Name,Is_Ad,Star_Rating,Num_Reviews,Price,Is_Bestseller,Num_Sales,Num_Basket,Description,Days_to_Arrival,Cost_Delivery,Returns_Accepted,Num_Images
0,"Happy birthday card, Bird birthday card, Natur...",279,1,5.0,2391,4.99,1.0,14319,20,Each piece at Chau Art is originally hand desi...,3.0,0.99,1,1
1,"Carousel Happy Birthday Card, Handmade Laser C...",1689,1,5.0,1728,5.99,1.0,15066,0,POSTAGE: UK - From 5 Sept 2020 all orders will...,6.0,2.95,1,1
2,FLOWER BIRTHDAY CARD | Birthday Card for Her |...,348,1,5.0,97,2.30,1.0,1041,20,Flower birthday card featuring a pink orchid o...,6.0,1.00,0,3
3,"Personalized Leather Card Holder, Black red bl...",991,1,4.5,1345,25.19,1.0,11375,20,▲ Croc embossed Genuine Leather Personalised C...,4.0,3.99,0,10
4,"Among Us Birthday Card - It's Your Birthday, S...",1982,1,5.0,24,2.95,1.0,843,20,"Among Us Birthday Card - Its Your Birthday, Se...",4.0,1.00,1,8


In [3]:
df['Star_Rating'].value_counts()

5.0    11087
4.5      215
4.0       11
3.5        3
1.0        1
Name: Star_Rating, dtype: int64

In [4]:
df= df[df['Star_Rating'] > 3.5]

In [5]:
df['Star_Rating'].value_counts()

5.0    11087
4.5      215
4.0       11
Name: Star_Rating, dtype: int64

In [6]:
df.columns

Index(['Title', 'Shop_Name', 'Is_Ad', 'Star_Rating', 'Num_Reviews', 'Price',
       'Is_Bestseller', 'Num_Sales', 'Num_Basket', 'Description',
       'Days_to_Arrival', 'Cost_Delivery', 'Returns_Accepted', 'Num_Images'],
      dtype='object')

In [42]:
df['Total_Cost_PC'] = df['Price'] + df['Cost_Delivery']
grouped = df.groupby('Shop_Name')[['Total_Cost_PC','Num_Sales']].max()
grouped.head()

,Total_Cost_PC,Num_Sales
Shop_Name,,
1,2.90,8098
2,4.00,14290
3,3.60,14397
5,3.49,36363
6,4.99,1732


In [47]:
by_shops = df.groupby('Shop_Name').agg({'Num_Sales':'max','Price':'mean', 'Total_Cost_PC':'mean','Is_Ad': 'mean', 'Star_Rating':'mean', 'Num_Reviews':'max','Num_Basket':'mean','Cost_Delivery':'mean','Days_to_Arrival': 'mean','Num_Images': 'mean', 'Returns_Accepted':'mean'}).sort_values('Num_Sales', ascending=False).reset_index()
by_shops['Total_Revenue'] = by_shops['Total_Cost_PC'] * by_shops['Num_Sales']
by_shops.head()

,Shop_Name,Num_Sales,Price,Total_Cost_PC,Is_Ad,Star_Rating,Num_Reviews,Num_Basket,Cost_Delivery,Days_to_Arrival,Num_Images,Returns_Accepted,Total_Revenue
0,776,128763,2.625,2.962222,0.0,5.0,1179,0.777778,0.337222,5.833333,2.944444,0.777778,381424.620000
1,1633,80788,4.300,5.290000,0.0,5.0,94,15.000000,0.990000,8.000000,9.000000,0.000000,427368.520000
2,362,80765,3.000,3.491538,0.0,5.0,550,2.692308,0.491538,5.769231,3.192308,0.615385,281994.103846
3,1337,80763,4.490,4.925000,0.0,5.0,476,6.500000,0.435000,5.750000,4.750000,0.250000,397757.775000
4,1751,80762,2.000,2.495000,0.0,5.0,119,7.500000,0.495000,8.500000,6.500000,0.500000,201501.190000
